In [268]:
import torch
import torchvision
import numpy as np
import os.path as osp
import os

In [269]:
CUDA = torch.cuda.is_available()

In [270]:
# GLOBALS
# vot_path = "../../../vot2016/"
vot_path = "/home/zabulskyy/Datasets/vot2016/"
# yolo_pred_path = "../yolo_predictions/extended"
yolo_pred_path = "/mnt/sshfs/YOLO-tracker/yolo_predictions/extended"
crit_vals = {
    "iou": 1,
    "dist2": 1,
#     "abs_pos": 1,
#     "rel_pos": 1,
    "cc": 100,  # class_correspondence
}
with open("../../yoloTorch/data/coco.names", "r") as f:
    classes = f.read().split("\n")[:-1]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [271]:
def read_csv(file="correlations.csv"):
    with open(file, 'r') as file:
        l = file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")[:-1]] for y in l]   
        return torch.tensor(l)
    

In [272]:
cm = read_csv()

In [273]:
def dist2(row1, row2):
    pass

def the_closest(row_to_compare, rows):
    pass

def iou(boxA, boxB):
    pass

def the_most_iou(row_to_compare, rows):
    pass

def interpolate(data):
    pass

def read_gt(vot_class, vot_path=vot_path, force_square=True):
    pass

def read_yolo_pred(vot_class, yolo_pred_path=yolo_pred_path, force_square=True):
    pass

def eval_energy(bb1, bb2, cv=crit_vals):
    pass

def class_correlations(class_probs1, class_probs2):
    pass

def eval_class_corr(cls1, cls2, correlations=cm):
    pass

In [274]:
def dist2(row1, row2):
    x11, y11, x12, y12 = row1[1:5] if len(row1) != 4 else row1[:]
    x21, y21, x22, y22 = row2[1:5] if len(row2) != 4 else row2[:]
    x1, y1 = (x11 + x12) / 2, (y11 + y12) / 2
    x2, y2 = (x21 + x22) / 2, (y21 + y22) / 2
    X = (x1 - x2) ** 2
    Y = (y1 - y2) ** 2
    res = float((X + Y) ** 0.5)
    return res
def the_closest(row_to_compare, rows):
    m, i = np.infty, 0
    for n, row in enumerate(rows):
        d = dist2(row, row_to_compare)
        if d < m:
            m = float(d)
            i = n
    return rows[i], i

In [275]:
def iou(boxA, boxB):
    boxA = boxA[1:5] if len(boxA) != 4 else boxA
    boxB = boxB[1:5] if len(boxB) != 4 else boxB

    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    res = interArea / float(boxAArea + boxBArea - interArea)
    return res
def the_most_iou(row_to_compare, rows):
    M, i = 0, 0
    for n, row in enumerate(rows):
        v = iou(row, row_to_compare)
        if v > M:
            M = float(v)
            i = n
    return rows[i], i

In [311]:
def generate_subframes(row1, row2, width):
    frame1, frame2 = int(row1[0]), int(row2[0])
    diff = frame1 - frame2
    if diff == 1:
        return None
    result = torch.zeros((frame2 - frame1 - 1, width))
    # result[:, -1] = row1[-1]
    step = (row1 - row2) / diff
    for frame_n in range(frame2 - frame1 - 1):
        result[frame_n] = row1 + (step) * (frame_n + 1)
    return result


def merge_frames(*rows):
    result = torch.zeros((1, 8))
    n = len(rows)
    for row in rows:
        result += row
    return result / n


def the_closest(row_to_compare, rows):
    def dist(row1, row2):
        x11, y11, x12, y12 = row1[1:5] if len(row1) != 4 else row1[:]
        x21, y21, x22, y22 = row2[1:5] if len(row2) != 4 else row2[:]
        x1, y1 = (x11 + x12) / 2, (y11 + y12) / 2
        x2, y2 = (x21 + x22) / 2, (y21 + y22) / 2
        X = (x1 - x2) ** 2
        Y = (y1 - y2) ** 2
        res = float((X + Y) ** 0.5)
        return res

    m, i = np.infty, 0
    for n, row in enumerate(rows):
        d = dist(row, row_to_compare)
        if d < m:
            m = float(d)
            i = n
    return rows[i], i

def fill_first(tensor):
    FIRST = tensor[0]
    for i in range(int(FIRST[0]) - 1, -1, -1):
        row = FIRST
        row[0] = i
        row = row.view((1, -1))
        tensor = torch.cat((row, tensor))
    return tensor


def fill_last(tensor, NUM_FRAMES):
    last = tensor[tensor[:, 0] != 0][-1]
    for i in range(int(last[0]) + 1, NUM_FRAMES):
        row = last
        row[0] = i
        row = row.view((1, -1))
        tensor[i] = row
    return tensor


def most_frequent_class(results):
    counts = np.bincount(results[:, -1])
    return np.argmax(counts)


def interpolate(data):
    """
    interpolate gaps
    choose one detection among multiple, by picking the closest one by the euclidean distance
    ignore class labels
    :param OUTPUT: tensor
    :param NUM_FRAMES:
    :param CUDA: bool - if cuda is available
    :return: interpolated tensor
    """
    OUTPUT = data["output"]
    NUM_FRAMES = data["num_frames"]
    CUDA = data["CUDA"]

    width = OUTPUT.size()[1]
    result = torch.zeros((NUM_FRAMES, width))
    if CUDA:
        result = result.cuda()

    if OUTPUT[0][0] != 0:
        OUTPUT = fill_first(OUTPUT)

    output_iter = 0
    res_iter = 0
    while res_iter != NUM_FRAMES:
        if (output_iter > len(OUTPUT) - 1):
            # break
            result = fill_last(result, NUM_FRAMES)
            break
        if output_iter == OUTPUT.shape[0] - 1:
            result[res_iter] = OUTPUT[output_iter]
            output_iter += 1
            res_iter += 1
            continue

        # next frame is interpolated
        if OUTPUT[output_iter][0] == OUTPUT[output_iter + 1][0] - 1:
            result[res_iter] = OUTPUT[output_iter]
            output_iter += 1
            res_iter += 1
            continue

        # missing detection on some frames, fill with missing
        if OUTPUT[output_iter][0] + 1 < OUTPUT[output_iter + 1][0]:
            subframes = generate_subframes(
                OUTPUT[output_iter], OUTPUT[output_iter + 1], width)
            result[res_iter] = OUTPUT[output_iter]
            for subframe in subframes:
                res_iter += 1
                result[res_iter] = subframe
            output_iter += 1
            res_iter += 1
            continue

        # this frame contains multiple detections, have to choose one
        if OUTPUT[output_iter][0] == OUTPUT[output_iter + 1][0]:
            frame = OUTPUT[output_iter][0]
            to_cut = OUTPUT[OUTPUT[:, 0] == frame]
            # closest, _ = the_closest(result[res_iter - 1], to_cut)
            # result[res_iter] = closest
            closest, _ = the_most_iou(result[res_iter - 1], to_cut)
            result[res_iter] = closest

            output_iter += len(to_cut)
            res_iter += 1

            if output_iter >= OUTPUT.shape[0] - 1:
                break

            # missing detection on some frames, fill with missing
            if OUTPUT[output_iter - 1][0] + 1 < OUTPUT[output_iter][0]:
                subframes = generate_subframes(
                    result[res_iter - 1], OUTPUT[output_iter], width)
                # result[res_iter] = OUTPUT[output_iter]
                for subframe in subframes:
                    result[res_iter] = subframe
                    res_iter += 1
            continue
    return result

In [309]:
def replace_first_frame(to_replace, OUTPUT):
    OUTPUT = OUTPUT[OUTPUT[:, 0] != 0]
    return torch.cat((to_replace.view((1, -1)), OUTPUT))

def pp_first(data):
    # detects the closest object to FIRST gt box and removes the rest on the FIRST frame
    CUDA = data["CUDA"]
    output = data["output"]
    first = data["first"]

    tcc, _ = the_most_iou(first, output)
    # only detections close to the true FIRST frame
    output = replace_first_frame(tcc, output)
    return output

In [278]:
def read_gt(vot_class, vot_path=vot_path, force_square=True):
    gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
    with open(gt_file, 'r') as file:
        l =file.read().split("\n")
        l.pop(-1)
        l = [[float(x) for x in y.split(",")] for y in l]
        if force_square:
            l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        return torch.tensor(l)
    
def read_full_gt(vot_path=vot_path, force_square=True):
    vot_classes = [x for x in os.listdir(vot_path) if not x.endswith(".txt")]
    X = list()
    for vot_class in vot_classes:
        gt_file = osp.join(vot_path, vot_class, "groundtruth.txt")
        with open(gt_file, 'r') as file:
            l =file.read().split("\n")
            l.pop(-1)
            l = [[float(x) for x in y.split(",")] for y in l]
            if force_square:
                l = [[min(m[::2]), min(m[1::2]), max(m[::2]), max(m[1::2])] for m in l]   
        X.append(l)
    return torch.tensor(X)

In [279]:
def read_yolo_pred(vot_class, yolo_pred_path=yolo_pred_path, pred_format="csv", first_len=True):
    yolo_pred_file = osp.join(yolo_pred_path, vot_class + "." + pred_format)
    with open(yolo_pred_file, 'r') as file:
        l =file.read().split("\n")
        l.pop(-1)
        im_len = -1
        if first_len:
            im_len = float(l.pop(0))
        l = [[float(x) for x in y.split(",")] for y in l]            
        return torch.tensor(l), im_len
    

In [280]:
# compute class correlation
def ccc(classes_probs):
    # classes_probs.shape = (bboxes, classes)
    corr_mat = torch.zeros((classes_probs.shape[1], classes_probs.shape[1], ))

    for cls in range(len(classes_probs[0])):
        col = classes_probs[:, cls].view((-1, 1))
        tX = col * classes_probs
        sX = torch.sum(tX, dim=0)
        corr_mat[cls] = sX
    return corr_mat

def corr(col1, col2):
    mean1, mean2 = torch.mean(col1), torch.mean(col2)
    cm1, cm2 = col1 - mean1, col2 - mean2
    top = sum(cm1 * cm2)
    bot = torch.sqrt(sum(cm1 ** 2) * sum(cm2 ** 2))
    return top / bot

# compute class correlation coefficient
def cccc(classes_probs):
    # classes_probs.shape = (bboxes, classes)
    corr_mat = torch.zeros((classes_probs.shape[1], classes_probs.shape[1], ))
    num_classes = len(classes_probs[0])
    for cls1 in range(num_classes):
        col1 = classes_probs[:, cls1].view((-1, 1))
        for cls2 in range(cls1 + 1, num_classes):
            print("{}.{} / {}".format(cls1+1, cls2+1, num_classes))
            col2 = classes_probs[:, cls2].view((-1, 1))
            _corr = corr(col1, col2)
#             _corr = np.corrcoef(col1, col2)
            corr_mat[cls1, cls2] = _corr
    return corr_mat


In [281]:
def read_all_yolo_preds(path=yolo_pred_path, pred_format="csv"):
    X = []
    for vot_class in os.listdir(path):
        yolo_pred_file = osp.join(path, vot_class)
        with open(yolo_pred_file, 'r') as file:
            l = file.read().split("\n")
            l.pop(-1)
            l.pop(0)
            im_len = -1
            for i in l:
                X.append([float(x) for x in i.split(",")])
    X = torch.tensor(X)
#     cls_probs = X[:, 6:]
    return X

In [282]:
def eval_single_class_corr(cls1 : int, cls2 : int, correlations=cm):
    return float(cm[cls1][cls2])

def eval_whole_class_corr(cls1, cls2, correlations=cm):
    # TODO: consider if this worth it
    # too complicated yet
    pass

In [283]:
def normalise(tensor):
    mean = torch.mean(tensor)
    std = torch.std(tensor)
    return (tensor - mean) / std

In [284]:
def write_csv(matrix, file="correlations.csv"):
    for i in matrix.tolist():
        for j in i:
            print(j, sep=", ", end=", ", file=open(file, "a+"))
        print(file=open(file, "a+"))

In [285]:
# class_i = classes.index("orange")
# cm_vals = torch.sort(cm[class_i], descending=True)[0]
# cm_idxs = torch.sort(cm[class_i], descending=True)[1]

# i = 0
# while i != 80:
#     print(classes[cm_idxs[i]], "\t\t" , float(cm_vals[i]))
#     i += 1
    

In [286]:
def eval_energy(bb1, bb2, cv=crit_vals):
    box1, box2 = bb1[1:5], bb2[1:5]
    cls1, cls2 = bb1[6: ], bb2[6: ]  # 5'th column is the precision of the bbox
    class1, class2 = torch.argmax(cls1), torch.argmax(cls2)
    
    _iou = iou(box1, box2)
    _cc = max(0, eval_single_class_corr(class1, class2))
    _dist2 = dist2(box1, box2)  # probably not
    
    return cv["iou"] * _iou, cv["cc"] * _cc, cv["dist2"] * _dist2

In [287]:
def cebtf(frame1, frame2):  # calc_energies_between_two_frames
    boxes = torch.zeros((len(frame1)))
    energies = torch.zeros((len(frame1)))
    for i in range(len(frame1)):
        max_energy = 0
        max_energy_index = 0
        for j in range(len(frame2)):
            energy = eval_energy(frame1[i], frame2[j])
            energy = energy[0] + energy[1]  # iou and class correspondence
            if energy > max_energy:
                max_energy = energy
                max_energy_index = j
        boxes[i] = max_energy_index
        energies[i] = max_energy
    return boxes, energies
          

In [287]:
total_res = dict()  

In [310]:
def get_first(vot_class):
    with open(osp.join(vot_path, vot_class, "groundtruth.txt"), 'r') as file:
        l = [float(x) for x in file.read().split("\n")[0].split(",")]
        X, Y = l[::2], l[1::2]
        l = [min(X), min(Y), max(X), max(Y)]
        l = torch.tensor([float(x) for x in l])
        return l

for vot_class in ["soccer1"]: #[x for x in os.listdir(vot_path) if not x.endswith(".txt")]:  

    print("\n", vot_class)
    yp, imlen = read_yolo_pred(vot_class)  # yolo predictions
    frames_with_detection = sorted(list(set(yp[:,0].tolist())))
    ffwdi = frames_with_detection[0]  # first_frame_with_detection_index
    first_boxes = yp[yp[:, 0] == ffwdi]
    path = torch.zeros((len(first_boxes), imlen))
    energies = torch.zeros((len(first_boxes), imlen))
    path.shape, energies.shape
    prev_boxes = first_boxes
    path[:, 0] = first_boxes[0][0]

    # calculate energies
    print("energies")
    for n, i in enumerate(frames_with_detection[:]):
        if n % 100 == 99:
            print(str(n / len(frames_with_detection[1:]) * 100)[:4], end="% ")
        i_boxes = yp[yp[:, 0] == i]
        boxes, energy = cebtf(prev_boxes, i_boxes)
        path[:, int(n)] = boxes
        energies[:, int(n)] = energy
        prev_boxes = i_boxes[boxes.long()]
    # path, energies
    verdict = torch.sum(energies, dim=1)

    first_bb_candidates = (verdict == torch.max(verdict)).nonzero().view(-1).tolist()
    res_i = path[first_bb_candidates].view(-1).int()
    len(res_i), len(frames_with_detection)
    res = torch.zeros((len(frames_with_detection), 5))

    # prepare for interpolation
    for n, i in enumerate(frames_with_detection[:]):
        cfp = yp[yp[:, 0] == i]  # current_frame_predictions
        index = res_i[n]
        cp = cfp[index] # chosen_prediction
        res[int(n)] = cp[:5]
        
    data = dict()
    data["output"] = res
    data["num_frames"] = int(imlen)
    data["CUDA"] = CUDA
    data["first"] = get_first(vot_class)
    
    print("\ninterpolation")
    print(data["output"] )
    
    total_res[vot_class] = interpolate(data)[:, 1:].tolist()

    


 soccer1
energies
34.6% 69.5% 
interpolation
tensor([ 300.7100,  135.5500,  371.7500,  219.9700]) tensor([[   0.0000,  265.0144,    4.0887,  566.5678,  360.0000],
        [   2.0000,  251.9438,   16.2566,  575.1027,  353.0335],
        [   3.0000,  243.1795,   28.1014,  577.6489,  353.6281],
        ...,
        [ 389.0000,  309.5993,   74.0018,  524.3060,  354.6740],
        [ 390.0000,  305.7798,   79.9999,  521.1733,  353.2141],
        [ 391.0000,  315.4599,   77.5337,  518.9969,  357.6635]])
tensor([[ 182.0000,  308.7520,  177.5748,  391.6583,  239.2658],
        [   2.0000,  251.9438,   16.2566,  575.1027,  353.0335],
        [   3.0000,  243.1795,   28.1014,  577.6489,  353.6281],
        ...,
        [ 389.0000,  309.5993,   74.0018,  524.3060,  354.6740],
        [ 390.0000,  305.7798,   79.9999,  521.1733,  353.2141],
        [ 391.0000,  315.4599,   77.5337,  518.9969,  357.6635]])


IndexError: index 392 is out of bounds for dimension 0 with size 392

In [301]:
data["first"]

tensor([ 300.7100,  135.5500,  371.7500,  219.9700])

In [298]:
total_res.keys()

dict_keys(['blanket', 'bolt2', 'traffic', 'ball1', 'leaves', 'pedestrian1', 'singer3', 'helicopter', 'godfather'])

torch.Size([12153, 86])

In [253]:
def save(res, folder, ext="csv", nm_fr=None):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for name in res:
        if nm_fr is not None:
            print(nm_fr[name], file=open(osp.join(folder, name + "." + ext), 'w+'))
            print("\n".join([",".join([str(a) for a in x]) for x in res[name]]),
                  file=open(osp.join(folder, name + "." + ext), 'a'))
        else:
            print("\n".join([",".join([str(a) for a in x]) for x in res[name]]),
                  file=open(osp.join(folder, name + "." + ext), 'w+'))


In [ ]:
save(total_res, "first_results_more_cc")

tensor([[   0.0000,  294.8584,  360.7994,  379.0862,  575.9958],
        [   1.0000,  303.0713,  359.6979,  385.3262,  576.0000],
        [   2.0000,  307.9651,  358.7937,  388.3065,  575.8509],
        ...,
        [ 399.0000,  286.9731,  300.8241,  337.3255,  400.4477],
        [ 400.0000,  281.9034,  307.2825,  335.9633,  401.4847],
        [ 401.0000,  278.1752,  309.6931,  332.9856,  396.9450]])

TypeError: pp_first() missing 1 required positional argument: 'data'